In [ ]:
import pandas as pd
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

gdpdf = pd.read_csv('Raw_Data\\Economical\\gpd_data_incurrentUSD.csv')
gdpdf = gdpdf.drop(['Country Name','Indicator Name','Indicator Code'], axis=1)
gdpdf = gdpdf.set_index('Country Code')
country_codes = pd.read_csv('Raw_data\\country_codes.csv')
gdpdf = gdpdf[gdpdf.index.isin(country_codes['alpha-3'])]
gdpdf = gdpdf.map(lambda x: x / 1e6 if x is not None else x)
gdpdf = gdpdf.round(2)
cols_to_drop = [col for col in gdpdf.columns if int(col) < 2002]
gdpdf = gdpdf.drop(columns=cols_to_drop)
gdpdf.columns = gdpdf.columns.astype(int)

co2ktdf = pd.read_csv('Raw_Data\\Environmental\\CO2_Emissions_ByCountry_KT.csv')
co2ktdf = co2ktdf.drop(['Series Name','Series Code','Country Name'], axis=1)
col_names = co2ktdf.columns.tolist()
col_names[1:] = [name[:4] for name in col_names[1:]]
co2ktdf.columns = col_names
co2ktdf = co2ktdf.set_index('Country Code')
co2ktdf = co2ktdf[co2ktdf.index.isin(country_codes['alpha-3'])]
co2ktdf = co2ktdf.apply(pd.to_numeric, errors='coerce')
cols_to_drop = [col for col in co2ktdf.columns if int(col) < 2002]
co2ktdf = co2ktdf.drop(columns=cols_to_drop)
co2ktdf.columns = co2ktdf.columns.astype(int)

lawindex = pd.read_csv('Raw_Data\\Legal\\Law_Index.csv')
lawindex = lawindex.drop(['Country Name','Indicator Name','Indicator Code'], axis=1)
lawindex = lawindex.set_index('Country Code')
lawindex = lawindex[lawindex.index.isin(country_codes['alpha-3'])]
cols_to_drop = [col for col in lawindex.columns if int(col) < 2002]
lawindex = lawindex.drop(columns=cols_to_drop)
lawindex.columns = lawindex.columns.astype(int)

politicalstab = pd.read_csv('Raw_Data\\Political\\political_stability_index.csv')
politicalstab = politicalstab.drop(['Series Name','Series Code','Country Name'], axis=1)
col_names = politicalstab.columns.tolist()
col_names[1:] = [name[:4] for name in col_names[1:]]
politicalstab.columns = col_names
politicalstab = politicalstab.set_index('Country Code')
politicalstab = politicalstab[politicalstab.index.isin(country_codes['alpha-3'])]
politicalstab = politicalstab.apply(pd.to_numeric, errors='coerce')
cols_to_drop = [col for col in politicalstab.columns if int(col) < 2002]
politicalstab = politicalstab.drop(columns=cols_to_drop)
politicalstab.columns = politicalstab.columns.astype(int)

ginindex = pd.read_csv('Raw_Data\\Social\\Gini_Index_(0good-100bad).csv')
ginindex = ginindex.drop(['Country Name','Indicator Name','Indicator Code'], axis=1)
ginindex = ginindex.set_index('Country Code')
ginindex = ginindex[ginindex.index.isin(country_codes['alpha-3'])]
cols_to_drop = [col for col in ginindex.columns if int(col) < 2002]
ginindex = ginindex.drop(columns=cols_to_drop)
ginindex.columns = ginindex.columns.astype(int)

techexpenditure1 = pd.read_csv('Raw_Data\\Technological\\API_GB.XPD.RSDV.GD.ZS_DS2_en_csv_v2_4.csv')
techexpenditure1 = techexpenditure1.drop(['Country Name','Indicator Name','Indicator Code'], axis=1)
techexpenditure1 = techexpenditure1.set_index('Country Code')
techexpenditure1 = techexpenditure1[techexpenditure1.index.isin(country_codes['alpha-3'])]
cols_to_drop = [col for col in techexpenditure1.columns if int(col) < 2002]
techexpenditure1 = techexpenditure1.drop(columns=cols_to_drop)
techexpenditure1.columns = techexpenditure1.columns.astype(int)
techexpenditure = techexpenditure1 * gdpdf


# Step 1
# Fetches the country data series from the dataframes and returns them as a list of series
def get_country_serie_list(country_code):
    countrydf1 = gdpdf.loc[country_code].rename('Gdp'+' '+country_code)
    countrydf2 = co2ktdf.loc[country_code].rename('CO2'+' '+country_code)
    # countrydf6 = techexpenditure.loc[country_code].rename('Techexpenditure'+' '+country_code) 
    countrydf3 = lawindex.loc[country_code].rename('Lawindex'+' '+country_code)
    countrydf4 = politicalstab.loc[country_code].rename('Politicalstab'+' '+country_code)
    countrydf5 = ginindex.loc[country_code].rename('Ginindex'+' '+country_code)
    
    country_serie_list = [countrydf1, countrydf2, countrydf3, countrydf4, countrydf5]
    return country_serie_list

# Step 2
# This function shortens the list of series to equal lenght aligned with the shortest serie
def shorten_series(series_list):
    first_valid_indices = [s.first_valid_index() for s in series_list]
    max_index = max(first_valid_indices)
    shortened_series_list = [s.loc[max_index:] for s in series_list]
    return shortened_series_list

# Step 3
# Change method if required!
def interpolator(serie_list):
    interpolated_serie_list = [serie.interpolate(method='linear') for serie in serie_list]
    return interpolated_serie_list

# Step 4
# Detrend the first two series in the list (which are gdp and co2)
def detrend_first_two(series_list):
    detrended_series_list = series_list.copy()
    for i in range(2):  # Loop over the first two series
        detrended_array = signal.detrend(series_list[i].values)
        detrended_series_list[i] = pd.Series(detrended_array, 
                                             index=series_list[i].index, 
                                             name=series_list[i].name)
        return detrended_series_list

# Step 5
# This function scales the series in the list
def scale_series_list(series_list):
    scaler = MinMaxScaler()
    scaled_series_list = []  
    for df_serie in series_list:
        # Scale the data
        scaled_serie = pd.Series(scaler.fit_transform(df_serie.values.reshape(-1, 1)).flatten(), index=df_serie.index)
        scaled_serie.name = df_serie.name  # Preserve the name
        scaled_series_list.append(scaled_serie)
    
    return scaled_series_list



def plot_with_trendlines(df):
    """
    Plots each column in the DataFrame as a subplot with a trendline and displays the trendline equation.
    
    Parameters:
    df (DataFrame): The DataFrame containing the data to plot.
    """
    fig, axes = plt.subplots(nrows=len(df.columns), ncols=1, figsize=(6, 10))
    
    for i, col in enumerate(df.columns):
        # Plot the original data
        df[col].plot(style='o-', ax=axes[i], title=col)
        
        # Calculate the trendline
        z = np.polyfit(df.index, df[col], 1)
        p = np.poly1d(z)
        
        # Get the trendline equation as a string
        trendline_eq = f"y = {z[0]:.2f}x + {z[1]:.2f}"
        
        # Plot the trendline
        plt.sca(axes[i])
        plt.plot(df.index, p(df.index), "r--", label='Trend')
        
        # Set x-axis ticks as integers with intervals of 3
        axes[i].set_xticks(np.arange(min(df.index), max(df.index)+1, 3))
        
        # Annotate the trendline equation
        plt.text(0.05, 0.95, trendline_eq, transform=axes[i].transAxes, fontsize=9,
                 verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor='red', facecolor='white'))
        
        # Add a legend
        # axes[i].legend()
    
    plt.tight_layout()
    plt.show()

# Example usage:
# plot_with_trendlines(fin_table_4)



from scipy.stats import shapiro, kstest, norm
import matplotlib.pyplot as plt
import statsmodels.api as sm

def assess_normality(df):
    """
    Performs Shapiro-Wilk and Kolmogorov-Smirnov tests on each column of the DataFrame.
    Also generates Q-Q plots and histograms for visual assessment of normality.
    
    Parameters:
    df (DataFrame): The DataFrame containing the data to test.
    """
    results = {}
    for column in df.columns:
        # Shapiro-Wilk Test
        stat_shapiro, p_value_shapiro = shapiro(df[column])
        # Kolmogorov-Smirnov Test
        stat_ks, p_value_ks = kstest(df[column], 'norm', args=(df[column].mean(), df[column].std()))
        
        results[column] = {
            'Shapiro-Wilk Stat': stat_shapiro,
            'Shapiro-Wilk P-Value': p_value_shapiro,
            'Kolmogorov-Smirnov Stat': stat_ks,
            'Kolmogorov-Smirnov P-Value': p_value_ks
        }
        
        # Q-Q plot
        sm.qqplot(df[column], line='s')
        plt.title(f"Q-Q Plot for {column}")
        plt.show()
        
        # Histogram
        #plt.figure()
        #df[column].hist(bins=20, edgecolor='black')
        #plt.title(f"Histogram for {column}")
        #plt.show()
    
    return pd.DataFrame(results)

# Example usage:
# results_df = assess_normality(your_dataframe)
# print(results_df)

In [ ]:
display(techexpenditure1)

In [ ]:
total_non_null = gdpdf.count().sum()
print("Total non-null values in gdpdf dataframe:", total_non_null)

total_non_null = co2ktdf.count().sum()
print("Total non-null values in co2ktdf dataframe:", total_non_null)

total_non_null = lawindex.count().sum()
print("Total non-null values in lawindex dataframe:", total_non_null)

total_non_null = politicalstab.count().sum()
print("Total non-null values in politicalstab dataframe:", total_non_null)

total_non_null = ginindex.count().sum()
print("Total non-null values in ginindex dataframe:", total_non_null)

total_non_null = techexpenditure.count().sum()
print("Total non-null values in techexpenditure dataframe:", total_non_null)

row_count = gdpdf.shape[0]
print("Row count:", row_count)

display(gdpdf)

In [ ]:
fin_code = 'USA'
fin_table_list = get_country_serie_list(fin_code)
fin_table = pd.concat(fin_table_list, axis=1)
fin_table.index.name = 'Year' 
display(fin_table)

fin_table_list = shorten_series(fin_table_list)
fin_table_2 = pd.concat(fin_table_list, axis=1)
fin_table_2.index.name = 'Year'
display(fin_table_2)


fin_table_list = interpolator(fin_table_list)
fin_table_3 = pd.concat(fin_table_list, axis=1)
fin_table_3.index.name = 'Year'
display(fin_table_3)
plot_with_trendlines(fin_table_3)

fin_table_list = detrend_first_two(fin_table_list)
fin_table_4 = pd.concat(fin_table_list, axis=1)
fin_table_4.index.name = 'Year'
display(fin_table_4)


fin_table_list = scale_series_list(fin_table_list)
fin_table_5 = pd.concat(fin_table_list, axis=1)
fin_table_5.index.name = 'Year'
display(fin_table_5)

results_df = assess_normality(fin_table_5)
display(results_df.tail(1))

ROUTINE CHECK

In [ ]:
country_code_2 = 'FIN'
country_series_list = scale_series_list(detrend_first_two(interpolator(shorten_series(get_country_serie_list(country_code_2)))))
country_dfusa = pd.concat(country_series_list, axis=1)
display(country_dfusa)
country_dfusa.plot(subplots=True, figsize=(20, 20))

print(country_dfusa.index[0])

ROUTINE CHECK 2

In [1]:
country_code = 'USA'
country_series_list = scale_series_list(detrend_first_two((shorten_series(get_country_serie_list(country_code)))))
country_df = pd.concat(country_series_list, axis=1)

print("First 5 rows:")
print(country_df.head())
print("\nLast 5 rows:")
print(country_df.tail())

country_df.plot(subplots=False, figsize=(18, 5),style=['o-','o-','o-','o-','o-'], title=country_code)

country_fin_matrix = country_df.corr(method='pearson')

plt.figure(figsize=(7, 5))
sns.heatmap(country_fin_matrix, annot=True, cmap='coolwarm')
plt.show()

NameError: name 'scale_series_list' is not defined

In [ ]:
correlation_matrix_kendall = country_df.corr(method='kendall')
correlation_matrix_spearman = country_df.corr(method='spearman')
correlation_matrix_pearson = country_df.corr(method='pearson')

In [ ]:
correlation_matrix_list = [correlation_matrix_kendall, correlation_matrix_spearman, correlation_matrix_pearson]
correlation_methods = ['Kendall', 'Spearman', 'Pearson']

for i, correlation_matrix in enumerate(correlation_matrix_list):
    plt.figure(figsize=(7, 5))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    plt.title(f'Correlation Matrix ({correlation_methods[i]})')
    plt.show()

In [ ]:
alpha3_codes = country_codes['alpha-3'].tolist()
failed_codes = []  # List to keep track of failed alpha-3 codes

display(alpha3_codes)

dataframes = []  # List to store series converted into dataframes
failed_codes = []  # List to keep track of failed alpha-3 codes

for country_code in alpha3_codes:
    try:
        country_series_list = scale_series_list(detrend_first_two(interpolator(shorten_series(get_country_serie_list(country_code)))))
        country_df = pd.concat(country_series_list, axis=1)
        print(country_df.columns[0], country_df.index[0])
        dataframes.append(country_df)  # Append the dataframe to the list
    except Exception as e:
        print(f"Failed to process alpha-3 code: {country_code}. Error: {str(e)}")
        failed_codes.append(country_code)

print(f"Failed alpha-3 codes: {failed_codes}")

display(len(dataframes))

alpha3_codes = [code for code in alpha3_codes if code not in failed_codes]

In [ ]:
print(len(failed_codes))
print("failed in")
print(len(alpha3_codes))
# trust me this is not bad not all apha-3 codes are existing as countries

In [ ]:
import openpyxl
correlation_matrices = []

for df in dataframes:
    corr_matrix = df.corr(method = 'pearson')
    correlation_matrices.append(corr_matrix)

index_list = []
values_list = []

# Iterate over the correlation matrices
for corr_matrix in correlation_matrices:
    # Iterate over the items in the matrix
    for pair, value in corr_matrix.unstack().items():
        # Add the pair to the index list and the value to the values list
        index_list.append(pair)
        values_list.append(value)

# Create a MultiIndex from the index list
index = pd.MultiIndex.from_tuples(index_list, names=['Variable 1', 'Variable 2'])

# Create a Series with the MultiIndex and the values list
s = pd.Series(values_list, index=index)

display(s)

In [ ]:
mask1 = s != 1


# Apply the masks to the Series
filtered_s = s[mask1] # filtered out values = 1
# filtered_s = filtered_s[filtered_s.index.get_level_values('Variable 1') < filtered_s.index.get_level_values('Variable 2')]

extract = filtered_s.sort_values(ascending=False)

extract.to_excel('outputs\\correlation_matrix.xlsx')

display(extract)

In [ ]:
income_alpha3 = pd.read_csv('world-bank-income-groups.csv')
income_alpha3_filtered = income_alpha3[income_alpha3['Code'].isin(alpha3_codes)]
income_alpha3_filtered = income_alpha3_filtered[income_alpha3_filtered['Year'] == 2020]
income_alpha3_filtered = income_alpha3_filtered[income_alpha3_filtered['Code'].isin(alpha3_codes)]
income_alpha3_filtered = income_alpha3_filtered.reset_index(drop=True)
display(income_alpha3_filtered)

In [ ]:
display(extract)
extract_1 = extract.reset_index()
extract_1 = extract_1.rename(columns={0: 'Values'})
display(extract_1)

In [ ]:
extract_1['Code'] = extract_1['Variable 2'].str[-3:]
merged_df = extract_1.merge(income_alpha3_filtered[['Code', 'Entity',"World Bank's income classification"]], on='Code', how='left')
merged_df['Variable 1'] = merged_df['Variable 1'].str.split(' ', expand=True)[0]
merged_df['Variable 2'] = merged_df['Variable 2'].str.split(' ', expand=True)[0]
display(merged_df)
merged_df.to_excel('outputs\\correlation_matrix_with_income_groups.xlsx')

In [ ]:
for_clusters = pd.read_csv('for_clustering_correlations.csv')
display(for_clusters)
for_clusters = for_clusters.dropna()
display(for_clusters)
for_clusters = for_clusters.set_index('Country Code', drop=True)
display(for_clusters)
print(for_clusters.index[0])

In [ ]:

bins = [-1,-0.799,-0.599,-0.399,-0.199,0, 0.199, 0.399, 0.599, 0.799, 1.000]  # Define your bin edges
labels = [-5,-4,-3,-2,-1,1,2,3,4,5]  # Define labels for the bins
# 'Very Strong Inverse', 'Strong Inverse','Moderate Inverse', 'Low Inverse','Very Low Inverse', 'Very Low','Low','Moderate','Strong','Very Strong'
# List of columns you want to bin
columns_to_bin = for_clusters[0:9]

binned_df = pd.DataFrame()

# Apply binning to each column
for column in columns_to_bin:
    binned_df[f'binned_{column}'] = pd.cut(for_clusters[column], bins=bins, labels=labels)


binned_df = binned_df.merge(income_alpha3_filtered[['Code', "World Bank's income classification"]],
                            left_index=True,
                            right_on='Code',
                            how='left')
binned_df.set_index('Code', inplace=True)

mapping = {'Low-income countries': 1, 'Lower-middle-income countries': 2, 'Upper-middle-income countries': 3, 'High-income countries': 4}
binned_df["World Bank's income classification"] = binned_df["World Bank's income classification"].replace(mapping)


binned_df.dropna(inplace=True)
display(binned_df)


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4).fit(binned_df)

binned_df['Cluster'] = kmeans.predict(binned_df)

display(binned_df)

binned_df.to_excel('outputs\\binned_clusterlabeled_df.xlsx')

In [ ]:
display(cluster_comparison["World Bank's income classification"].dtype)

cluster_comparison_clean = cluster_comparison.dropna()

cluster_comparison_clean["World Bank's income classification"] = cluster_comparison_clean["World Bank's income classification"].astype(int)

display(cluster_comparison_clean)

cluster_comparison_clean.groupby(['Cluster', "World Bank's income classification"]).size().unstack().plot(kind='bar', stacked=True)
cluster_comparison_clean.groupby(["World Bank's income classification",'Cluster' ]).size().unstack().plot(kind='bar', stacked=True)
cluster_comparison_clean.groupby(['Cluster','Cluster' ]).size().unstack().plot(kind='bar', stacked=True)
cluster_comparison_clean.groupby(["World Bank's income classification","World Bank's income classification" ]).size().unstack().plot(kind='bar', stacked=True)

